In [36]:
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

import boto3

account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name

In [142]:
suffix = "com"
if region_name.startswith('cn'):
    print(region_name)

cn-nor


In [78]:
model_name = None
image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/aquila-chat-7b-inference-api:latest".format(
    account_id, region_name
)
base_name = sagemaker.utils.base_name_from_image(image_uri)

## 2 下载模型文件
### （方式1）去官网下载:
https://model.baai.ac.cn/model-detail/100101

模型下载后，放入aquilachat-7b 目录下，并将这个目录打包成model.tar.gz文件  

`tar czvf model.tar.gz aquilachat-7b`  
例如：  
`aws s3 cp model.tar.gz s3://sagemaker-{region}-{accountid}/model/aquila-chat-7b/model.tar.gz`  

### （方式2）从临时S3拉取，需要提前联系开权限
### 联系AWS开通临时s3权限，直接使用已经打包好的s3://sagemaker-us-east-2-946277762357/model/aquila-chat-7b/model.tar.gz

In [141]:
## 方式1
# model_data = f"s3://{bucket}/model/aquila-chat-7b/model.tar.gz"

##方式2
model_data = "s3://sagemaker-us-east-2-946277762357/model/aquila-chat-7b/model.tar.gz"

print(model_data)

s3://sagemaker-us-east-2-946277762357/model/aquila-chat-7b/model.tar.gz


In [125]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

model = Model(
    name=model_name,
    model_data=model_data,
    role=role,
    image_uri=image_uri,
    predictor_cls=Predictor,
)

In [126]:
instance_type = "ml.g5.4xlarge"
instance_count = 1

In [127]:
predictor = model.deploy(
    instance_type=instance_type,
    initial_instance_count=instance_count,
    container_startup_health_check_timeout=1200,
)

-----------------!

In [132]:
print(predictor.endpoint_name)

aquila-chat-7b-inference-api-2023-06-19-09-33-46-180


In [133]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

inputs = {
    "inputs": "user:你好？",
    "max_length":200,
}

prediction = predictor.predict(inputs)

In [134]:
prediction['outputs']

'你好！有什么我可以为您效劳的吗？\n'

In [124]:
#删除模型节点
predictor.delete_endpoint()